In [1]:
import sympy as sp
import numpy as np
from tensorflow.keras.models import load_model
import joblib

In [2]:

# Cargar el modelo y los escaladores
model = load_model('../best_model_2.4367305027108534e-05.h5')
scaler_X = joblib.load('../minmax_scaler_X.pkl')
scaler_y = joblib.load('../minmax_scaler_y.pkl')

c:\Users\PcR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
# Extraer pesos y sesgos
W1, b1 = model.layers[0].get_weights()
W2, b2 = model.layers[1].get_weights()
W3, b3 = model.layers[2].get_weights()

# Convertir a matrices SymPy
W1_sym = sp.Matrix(W1)
b1_sym = sp.Matrix(b1)
W2_sym = sp.Matrix(W2)
b2_sym = sp.Matrix(b2)
W3_sym = sp.Matrix(W3)
b3_sym = sp.Matrix(b3)

In [4]:
# Convertir los valores del scaler a símbolos
scaler_X_min = sp.Matrix(scaler_X.min_)
scaler_X_scale = sp.Matrix(scaler_X.scale_)
scaler_y_min = sp.sympify(scaler_y.min_[0])
scaler_y_scale = sp.sympify(scaler_y.scale_[0])

In [5]:
def normalize(x):
    return (x - scaler_X_min).multiply_elementwise(1 / scaler_X_scale)

def denormalize(y):
    return y * scaler_y_scale + scaler_y_min

# Definir funciones de activación simbólicas
def relu(x):
    return sp.Piecewise((0, x <= 0), (x, x > 0))

def sigmoid(x):
    return 1 / (1 + sp.exp(-x))

In [7]:
# Propagación hacia adelante simbólica
x_norm = normalize(x)
z1 = W1_sym.T * x_norm + b1_sym
a1 = z1.applyfunc(relu)

z2 = W2_sym.T * a1 + b2_sym
a2 = z2.applyfunc(sigmoid)

z3 = W3_sym.T * a2 + b3_sym
output_norm = z3

# Desnormalizar la salida
output = denormalize(output_norm[0])

# Definir la función simbólica completa
kGa_symbolic = sp.Lambda((y1, G_, wt_mea, z, T_gas, L_), output)

TypeError: unsupported operand type(s) for /: 'One' and 'MutableDenseMatrix'

In [ ]:
print("Función simbólica creada con normalización y desnormalización.")
print("Ejemplo de uso:")
print("kGa_symbolic(y1, G_, wt_mea, z, T_gas, L_)")

In [ ]:
# Función para convertir la expresión simbólica a una función numérica
def kGa_numeric(x):
    return float(kGa_symbolic(*x).evalf())